In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials, SpotifyOAuth

import os
import pandas as pd
import numpy as np
from pathlib import Path
import time
import matplotlib.pyplot as plt
from matplotlib.widgets import Cursor

import ipywidgets as widgets
from savify import Savify
from savify.types import Type, Format, Quality
from savify.utils import PathHolder
import logging

from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN
from scipy.cluster.hierarchy import dendrogram
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.neighbors import NearestNeighbors

from tqdm import tqdm
from IPython.display import display
import plotly.graph_objects as go
import plotly.subplots

from PyDMXControl.controllers import OpenDMXController
from PyDMXControl.profiles.Generic import Dimmer, Custom

import sounddevice as sd
from tqdm import tqdm
import threading
import matplotlib.pyplot as plt


%matplotlib inline

# get the client id and client secret from the text file
with open('spotify_credentials.txt') as f:
    client_id = f.readline().strip().split(' ')[1]
    client_secret = f.readline().strip().split(' ')[1]
    uri = f.readline().split(' ')[1][:-1]

uri = 'http://localhost:8000'

# set the environment variables
os.environ['SPOTIPY_CLIENT_ID'], os.environ['SPOTIFY_CLIENT_ID'] = client_id, client_id
os.environ['SPOTIPY_CLIENT_SECRET'], os.environ['SPOTIFY_CLIENT_SECRET'] = client_secret, client_secret
os.environ['SPOTIPY_REDIRECT_URI'], os.environ['SPOTIFY_REDIRECT_URI'] = uri, uri

#sp = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials())

auth_manager = SpotifyClientCredentials()
sp = spotipy.Spotify(auth_manager=auth_manager)

# just add all the scopes
scopes = ['user-library-read',
            'user-read-recently-played',
            'user-top-read',
            'user-follow-read',
            'user-read-playback-position',
            'user-read-playback-state',
            'user-read-currently-playing',
            'user-modify-playback-state',
            'user-read-private',
            'playlist-read-private',
            'playlist-read-collaborative',
            'playlist-modify-public',
            'playlist-modify-private']

username = '1260351083'

token = spotipy.util.prompt_for_user_token(username, scopes)

if token:
    sp = spotipy.Spotify(auth=token)
    saved_tracks_resp = sp.current_user_saved_tracks(limit=50)
else:
    print('Couldn\'t get token for that username')

path_holder = PathHolder(downloads_path='downloads')
logger = logging.getLogger('savify')
s = Savify(path_holder=path_holder, logger=logger, download_format=Format.MP3)

user = sp.user(username)
sp.user = user

In [ ]:
def get_playlist_df(playlist_id):
    results = sp.playlist_tracks(playlist_id)
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])

    # convert this into a dataframe with the audio features, song name, artist, id, and link
    features = []
    for track in tracks:
        feature_data = sp.audio_features(track['track']['id'])[0]
        feature_data['name'] = track['track']['name']
        feature_data['artist'] = track['track']['artists'][0]['name']
        feature_data['id'] = track['track']['id']
        feature_data['link'] = track['track']['external_urls']['spotify']
        feature_data['popularity'] = track['track']['popularity']
        feature_data['genres'] = sp.artist(track['track']['artists'][0]['id'])['genres']
        features.append(feature_data)

    out = pd.DataFrame(features)
    # drop "type" and "uri" columns
    out.drop(columns=['type', 'uri'], inplace=True)
    return out


elm_tracks = get_playlist_df('spotify:playlist:1nNxmkhYsOpa6Ux9MFJLoc')

In [ ]:
class Pulser(object):
    def __init__(self):
        self._running = False
        self.last_song = None
        self.df = pd.read_csv('data.csv')

    def start(self):
        self._running = True
        self.current_song = self.get_current_song()

    def get_current_song(self):
        self.current_song = sp.current_user_playing_track()['item']['id']
    
    def get_scene(self, song_id):
        # check if song is in the dataframe
        if song_id in self.df['id'].values:
            return self.df[self.df['id'] == song_id]['scene'].values[0]
        else:
            # estimate the scene based on genre
    
    def terminate(self):
        self._running = False
    
    def run(self):
        while self._running:
            current_song = self.get_current_song()
            if current_song != self.last_song:
                self.last_song = current_song
                self.pulse()
            time.sleep(1)

